In [1]:
import numpy as np

from pynq import Overlay
import pynq.lib.dma
from pynq import Xlnk
from pynq import MMIO

import time
import random

ol = Overlay('/home/xilinx/jupyter_notebooks/smul/smul.bit') # check your path
ol.download() # it downloads your bit to FPGA

dma = ol.streamMul.smul_dma # creating a dma instance. Note that we packed smul and smul_dma into streamMul

# sadd_ip = MMIO(0x43c00000, 0x10000) # we got this IP from Address Editor
sadd_ip = ol.streamMul.smul.mmio

xlnk = Xlnk()

In [ ]:
length = 11

in_buffer = xlnk.cma_array(shape=(length,), dtype=np.int32) # input buffer
out_buffer = xlnk.cma_array(shape=(length,), dtype=np.int32) # output buffer

samples = random.sample(range(0, length), length)
np.copyto(in_buffer, samples) # copy samples to inout buffer

sadd_ip.write(0x10, length) # we got this address from Vivado source

t_start = time.time()
dma.sendchannel.transfer(in_buffer)
dma.recvchannel.transfer(out_buffer)
dma.sendchannel.wait() # wait for send channel
dma.recvchannel.wait() # wait for recv channel
t_stop = time.time()

in_buffer.close()
out_buffer.close()

print('Hardware execution time: ', t_stop-t_start)
for i in range(0, length):
    print('{}*2 = {}'.format(in_buffer[i], out_buffer[i]))

In [21]:
ol.ip_dict

{'streamMul/smul': {'addr_range': 65536,
  'fullpath': 'streamMul/smul',
  'gpio': {},
  'interrupts': {},
  'phys_addr': 1136656384,
  'state': None,
  'type': 'xilinx.com:hls:smul:1.0'},
 'streamMul/smul_dma': {'addr_range': 65536,
  'fullpath': 'streamMul/smul_dma',
  'gpio': {},
  'interrupts': {},
  'phys_addr': 1077936128,
  'state': None,
  'type': 'xilinx.com:ip:axi_dma:7.1'}}

In [ ]:
ol.streamMul.smul.